In [8]:
import os
import warnings
warnings.filterwarnings("ignore")
import torch
import pandas as pd

In [6]:
# movielens dataset
names = ['user_id', 'item_id', 'rating', 'timestamp']
data_dir = '../../data/MovieLens/ml-100k'

In [9]:
data = pd.read_csv(os.path.join(data_dir, 'u.data'), '\t', names=names, engine='python')

In [17]:
# user & item count
num_users = data.user_id.unique().shape[0]
num_items = data.item_id.unique().shape[0]

In [18]:
# summary
sparsity =1 - len(data) / (num_users * num_items)
sparsity

0.9369533063577546

In [19]:
# data split
train_items = {} # 记录每个uid, 所有评分records: k:uid, v:[(uid, iid, rating, time), ...]
test_items = {} # 记录每个uid, 最新评分records: k:uid, v:(iid, rating, time)
train_list = [] # 记录所有评分records (uid, iid, rating, time), ...按照uid、time从小到大排序
for line in data.itertuples():
    u, i , rating, time = line[1], line[2], line[3], line[4]
    train_items.setdefault(u, []).append((u, i, rating, time)) # 记录k-v: key是uid, v是(u,i,r,t)的records
    if u not in test_items or test_items[u][-1] < time: # 若这个uid未记录在test_items, 或该行的time比test_items所记录的更大
        test_items[u] = (i, rating, time) # 设定/更换test_items中这个uid的最新评分记录
for u in range(1, num_users+1): # uid就是1到num_users的自然数字
    train_list.extend( sorted(train_items[u], key=lambda k: k[3]) ) # 记录每一个uid的所有评分记录, 按time从小到大排序
test_data =  [(key, *value) for key, value in test_items.items()] # 分解test_items到test_data: [(uid, iid, rating, time), ...]
train_data = [item for item in train_list if item not in test_data] # 不在test_data里的评分记录, 是train_data
train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)